In [1]:
## com uma ia jogado 
import pygame as pyg 
import os
import random
import neat  

aiPLay = True 
generation = 0


screenWidth = 500 
screeHeight = 800


imagePipe = pyg.transform.scale2x(pyg.image.load(os.path.join('imgs', 'pipe.png')))
imageGround = pyg.transform.scale2x(pyg.image.load(os.path.join('imgs', 'base.png')))
imagemBackGround = pyg.transform.scale2x(pyg.image.load(os.path.join('imgs', 'bg.png')))
imagesCBird = [
    pyg.transform.scale2x(pyg.image.load(os.path.join('imgs', 'bird1.png'))),
    pyg.transform.scale2x(pyg.image.load(os.path.join('imgs', 'bird2.png'))),
    pyg.transform.scale2x(pyg.image.load(os.path.join('imgs', 'bird3.png'))),
]

pyg.font.init()
fontPoints = pyg.font.SysFont('arial', 50)

class birdClass:
    imgs = imagesCBird
    ##animação de rotação 
    maxRotation = 25
    rotationSpeed = 15
    animationTime = 5
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.angle = 0
        self.speed = -11
        self.height = self.y
        self.time = 0
        self. imageCount = 0
        self.images = self.imgs[0]
        
    def pular(self):
        self.speed = -10.5
        self.time = 1
        self.height = self.y
    
    def mover(self):
        ##calcula o displacement 
        self.time += 1
        displacement = 1.5 * (self.time ** 2) + self.speed * self.time
        
        # restringir o displacement
        if displacement > 16:
            displacement = 16
        elif displacement < 0:
            displacement -= 2
            
        self.y += displacement
        
        ## angle do bird
        if displacement < 0 or self.y < (self.height + 50):
            if self.angle < self.maxRotation:
                self.angle = self.maxRotation
                
        else:
            if self.angle > -90:
                self.angle -= self.rotationSpeed
        
                    
    def draw(self, screePlay):
        # definir qual image do bird vai usar
        self. imageCount += 1
        
        if self. imageCount < self.animationTime:
            self.images = self.imgs[0]
        elif self. imageCount < self.animationTime * 2:
            self.images = self.imgs[1]
        elif self. imageCount < self.animationTime * 3:
            self.images = self.imgs[2]
        elif self. imageCount < self.animationTime * 4:
            self.images = self.imgs[1]
        elif self. imageCount >= self.animationTime * 4 + 1:
            self.images = self.imgs[0]
            self. imageCount = 0
        # se o bird tiver caindo eu não vou bater asa
        if self.angle <= -80:
            self.images = self.imgs[1]
            self. imageCount = self.animationTime * 2
        
        # draw a image
        rotatedImage = pyg.transform.rotate(self.images, self.angle)
        centerPoint = self.images.get_rect(topleft = (self.x, self.y)).center
        retangler  = rotatedImage.get_rect(center = centerPoint)
        screePlay.blit(rotatedImage, retangler .topleft)
            
    def get_mask(self):
        return pyg.mask.from_surface(self.images)

class pipeClass:
    distance = 150
    speed = 10
    
    def __init__(self, x):
        self.x = x
        self.height = 0
        self.posTop = 0
        self.posBase = 0
        self.pipeTop = pyg.transform.flip(imagePipe, False, True)
        self.pipeBase = imagePipe
        self.passed = False
        self.setHeight()
        
    def setHeight(self):
        self.height = random.randrange(50, 450)
        self.posTop = self.height - self.pipeTop.get_height()
        self.posBase = self.height + self.distance
        
    def mover(self):
        self.x -= self.speed
        
    def draw(self, screePlay):
        screePlay.blit(self.pipeTop, (self.x, self.posTop))
        screePlay.blit(self.pipeBase, (self.x, self.posBase))
        
    def colidir(self, bird):
        birdMask = bird.get_mask()
        topoMask = pyg.mask.from_surface(self.pipeTop)
        baseMask = pyg.mask.from_surface(self.pipeBase)

        distanceTop = (self.x - bird.x, self.posTop - round(bird.y))
        distanceBase = (self.x - bird.x, self.posBase - round(bird.y))

        topPoint = birdMask.overlap(topoMask, distanceTop)
        basePoint = birdMask.overlap(baseMask, distanceBase)

        if basePoint or topPoint:
            return True 
        else:
            return False
class groundClass:
    speed = 5
    width = imageGround.get_width()
    image = imageGround

    def __init__(self, y):
        self.y = y
        self.x1 = 0
        self.x2 = self.width

    def mover(self):
        self.x1 -= self.speed
        self.x2 -= self.speed

        if self.x1 + self.width < 0:
            self.x1 = self.x2 + self.width
        if self.x2 + self.width < 0:
            self.x2 = self.x1 + self.width

    def draw(self, screePlay):
        screePlay.blit(self.image, (self.x1, self.y))
        screePlay.blit(self.image, (self.x2, self.y))


def drawScreen(screePlay, birds, canos, ground, points):
    screePlay.blit(imagemBackGround, (0, 0))
    for bird in birds:
        bird.draw(screePlay)
    for cano in canos:
        cano.draw(screePlay)

    textS = fontPoints.render(f"Pontuação: {points}", 1, (255, 255, 255))
    screePlay.blit(textS, (screenWidth - 10 - textS.get_width(), 10))
    
    if aiPLay:
        textS = fontPoints.render(f"geração: {generation}", 1, (255, 255, 255))
        screePlay.blit(textS, (10, 10))
    ground.draw(screePlay)
    pyg.display.update()


    
    
    
    
def main(genomes, settings):
    global generation
    generation += 1

    if aiPLay:
        netWorks = []
        listaGenoma = []
        birds = []
        for _, genome in genomes:
            netWork = neat.nn.FeedForwardNetwork.create(genome, settings)
            netWorks.append(netWork)
            genome.fitness = 0 
            listaGenoma.append(genome)
            birds.append(birdClass(230, 350))
            
    
    else:
        birds = [birdClass(230, 350)]
        
    ground = groundClass(730)
    canos = [pipeClass(700)]
    screePlay = pyg.display.set_mode((screenWidth, screeHeight))
    points = 0
    relogio = pyg.time.Clock()

    rodando = True
    while rodando:
        relogio.tick(30)

        # interação com o usuário
        for evento in pyg.event.get():
            if evento.type == pyg.QUIT:
                rodando = False
                pyg.quit()
                quit()
            
            if not aiPLay:
                if evento.type == pyg.KEYDOWN:
                    if evento.key == pyg.K_SPACE:
                        for bird in birds:
                            bird.pular()
                            
            
        
        indeceCano = 0
        if len(birds) > 0:
            if len(canos) > 1 and birds[0].x > (canos[0].x + canos[0].pipeTop.get_width()):
                indeceCano = 1
        else:
            rodando = False

        # mover as coisas
        for i, bird in enumerate(birds):
            bird.mover()
            ##aumenta a fitness do passarao
            listaGenoma[i].fitness += 0.1
            output = netWorks[i].activate((bird.y, 
                                        abs(bird.y - canos[indeceCano].height),
                                        abs(bird.y - canos[indeceCano].posBase)))
            ## ativação entre -1 e 1---> se o o utput for > 0.5 ele pula se não não pula 
            if output[0] > 0.5:
                bird.pular()
            
            
            
        ground.mover()

        adicionarCano = False
        removerCanos = []
        for cano in canos:
            for i, bird in enumerate(birds):
                if cano.colidir(bird):
                    birds.pop(i)
                    if aiPLay:
                        listaGenoma[i].fitness -= 1
                        listaGenoma.pop(i)
                        netWorks.pop(i)              
                                       
                if not cano.passed and bird.x > cano.x:
                    cano.passed = True
                    adicionarCano = True
            cano.mover()
            if cano.x + cano.pipeTop.get_width() < 0:
                removerCanos.append(cano)

        if adicionarCano:
            points += 1
            canos.append(pipeClass(600))
            for genome in listaGenoma:
                genome.fitness += 5
                                       
        for cano in removerCanos:
            canos.remove(cano)

        for i, bird in enumerate(birds):
            if (bird.y + bird.images.get_height()) > ground.y or bird.y < 0:
                birds.pop(i)
                if aiPLay:
                    listaGenoma.pop(i)
                    netWorks.pop(i)
        drawScreen(screePlay, birds, canos, ground, points)

                                       
def rodar(caminhoArquivo):
    config = neat.config.Config(neat.DefaultGenome,
                               neat.DefaultReproduction,
                               neat.DefaultSpeciesSet,
                               neat.DefaultStagnation,
                               caminhoArquivo)
                                       
    populacao = neat.Population(config)
    populacao.add_reporter(neat.StdOutReporter(True))
    populacao.add_reporter(neat.StatisticsReporter())
    if aiPLay:
        populacao.run(main, 50)
    else:
        main(None, None)

if __name__ == '__main__':
    caminho = os.path.dirname(___)
    caminhoArquivo = os.path.join(caminho,"config.txt")
    rodar(caminhoArquivo)                             

    

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html

 ****** Running generation 0 ****** 

Population's average fitness: 3.95900 stdev: 4.92285
Best fitness: 48.30000 - size: (1, 3) - species 1 - id 97
Average adjusted fitness: 0.042
Mean genetic distance 1.058, standard deviation 0.391
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100     48.3    0.042     0
Total extinctions: 0
Generation time: 8.792 sec

 ****** Running generation 1 ****** 

Population's average fitness: 4.89600 stdev: 8.27588
Best fitness: 80.40000 - size: (1, 3) - species 1 - id 97
Average adjusted fitness: 0.036
Mean genetic distance 1.306, standard deviation 0.536
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100     80.4    0.036     0
Total extinctions: 0
Generation tim

: 

SyntaxError: invalid syntax (3982307617.py, line 233)

error: display Surface quit